# Puzzle solving using an Object-Oriented interface

*Inspired by Raymond Hettinger's talk at PyCon 2019*

A number of puzzles can be solved by either depth first search or breadth first search.

We present a generic puzzle solver that works on a broad class of puzzles.

The core idea is that we need very few things to describe the puzzle in a way that a solver can find a solution:

1. We need an initial position (the unsolved puzzle)
2. We need a rule (typically an iterator) to generate all possible moves from a position.
3. We need to recognize the goal state.

Optionally, we can provide some extras:

1. a `__repr__` method to display the puzzle position in a way recognizable to us.
2. a `isgoal()` method to recognize the goal state, by default set to `self.goal`.

For example, the following puzzle is solved for you: note the `__iter__` method using the `yield` keyword to generate neighbouring positions.

<div class="alert alert-warning">
    <b>Exercice:</b> Two puzzles are given for you to solve: the n-queens problem and the Solitaire problem.<br/>
Write the corresponding method so that the solve() method does the tree-search for you.
</div>    
<div class="alert alert-success">
    <b>Information:</b> By default, a breadth-first search is called, which is the safest option when the shortest path solution needs to be found or when some paths have a potential to wander around infinitely (i.e. you can randomly twist a Rubik's cube all day and never come near a solution).<br/>
    You may want to switch to depth-first search for the two other puzzles with option <code>solve(depthFirst=True)</code>.
</div>    

If you get an `IndexError: pop from an empty deque` exception, this means you finished exploring the search without finding a solution: check your `isgoal()` function and that you did not miss anything during the iteration.

In [1]:
from puzzle import Puzzle


class JugFill(Puzzle):
    """Given a two empty jugs with 3 and 5 liter capacities and a full
       jug with 8 liters, find a sequence of pours leaving four liters
       in the two largest jugs.
    """

    # https://dioverdt.files.wordpress.com/2011/01/jugs-problem.gif

    pos = (0, 0, 8)

    capacity = (3, 5, 8)

    goal = (0, 4, 4)

    def __iter__(self):
        for i in range(len(self.pos)):
            for j in range(len(self.pos)):
                if i == j:
                    continue
                qty = min(self.pos[i], self.capacity[j] - self.pos[j])
                if not qty: #qty == 0
                    continue
                dup = list(self.pos)
                dup[i] -= qty
                dup[j] += qty
                yield JugFill(tuple(dup))


JugFill().solve()

[(0, 0, 8),
 (0, 5, 3),
 (3, 2, 3),
 (0, 2, 6),
 (2, 0, 6),
 (2, 5, 1),
 (3, 4, 1),
 (0, 4, 4)]

In [4]:
# %load solutions/nqueens.py
class NQueens(Puzzle):
    """
    - ♛ - - - - - -
    - - - ♛ - - - -
    - - - - - ♛ - -
    - - - - - - - ♛
    - - ♛ - - - - -
    ♛ - - - - - - -
    - - - - - - ♛ -
    - - - - ♛ - - -
    
    Given n=8 queens, place them on an n × n chessboard so that no two queen
    attach each other in line, column and diagonal.
    """

    n = 8
    pos = tuple()

    def isgoal(self):
        self.n == len(self.pos)

    def __repr__(self):
        n = len(self.pos)
        return "\n".join(
            [" ".join("♛" if i == self.pos[j] else "-" for i in range(n)) for j in range(n)]
        )

    def __iter__(self):
        j = len(self.pos)
        for i in range(self.n):
            for qi, qj in enumerate(self.pos):
                if i != qi and abs(qi - qj) != abs(i - j):
                    print(self.__repr__)
                    yield NQueens(tuple([*self.pos, i]))


print(NQueens())
NQueens().solve(depthFirst=True)


IndexError: pop from an empty deque

In [7]:
for qi, qj in enumerate(tuple()):
    print(f"{i},{j}")

In [ ]:
# %load solutions/solitaire.py
class Solitaire(Puzzle):
    """
        ·
       x x
      x x x
     x x x x
    x x x x x

    Starting from the following puzzle, jump the tees like checkers,
    one at a time, removing each one you jump, until only one remains.

    """

    pos = (0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

    def isgoal(self):
        return sum(self.pos) == 1

    def __repr__(self):
        return ""

    def __iter__(self):
        yield Solitaire(self.pos)
    
Solitaire().solve()